### 0. Import libraries

In [3]:
# Torch and Torchlightning imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch

# Imports for SandiaPV
from pvlib import irradiance, temperature, pvsystem
import random

# import utility modulues 
from model_energy_prediction import *
from sandiapv_energy_prediction import *


# 1. Define training class


In [4]:

class TrainEnergyPrediction(pl.LightningModule):
    def __init__(self,dynamic_feature_size, static_feature_size, hidden_size, learning_rate=1e-3):
        super().__init__()
        self.model = EnergyPredictionModel(dynamic_feature_size, static_feature_size, hidden_size)
        self.loss_fn = nn.MSELoss(size_average=None, reduce=None, reduction='mean')
        self.learning_rate = learning_rate
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch):
        x_dynamic,x_static, y = batch
        y_pred = self.model(x_dynamic,x_static)
        loss = self.loss_fn(y_pred, y)
        self.log('train_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

## 2. Prepare data

In [ ]:
weather_data = load_weather_data('dataset/historical_weather.csv')
weather_data.index = pd.to_datetime(weather_data['timestamp'])
    
# Define location
site_location = location.Location(latitude=52.52, longitude=13.4050, altitude=34, tz='Europe/Berlin')

# Generate random panels
seed(0)
panels = generate_random_panels(3)
# - variables for random panels (ranges etc.)

# Initialize the dataset
dataset = EnergyDataset(weather_data, panels)
    
# Initialize the data loader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    


#### 2.2 add real measurement data to the synthetic dataset

## 3. Plot dataset

## 4. Intitialize model and start training

In [ ]:
# Initialize the model
dynamic_feature_size = 3
static_feature_size = 5
hidden_size = 64
model = EnergyPredictionModel(dynamic_feature_size, static_feature_size, hidden_size)
    
# Initialize the Lightning module
train_module = TrainEnergyPrediction(model)
    
# Initialize the Lightning Trainer
trainer = pl.Trainer(max_epochs=10)
    
# Fit the model to the training data
trainer.fit(train_module, data_loader)

#### 4.2 work towrads probabalistic model

## 5. Evaluate model